# Simple binary classification problem utilizing convolutional neural networks 

## Import libraries 

In [4]:
# Import libraries. 
from __future__ import print_function
import os
import sys
#os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu0,floatX=float32,optimizer=fast_compile'
#os.environ['KERAS_BACKEND'] = 'theano'
# """
# os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu3,floatX=float32,optimizer=fast_compile'
# os.environ['KERAS_BACKEND'] = 'theano'

# In case you want to select a graphic card (i the above code i set the 3rd graphic card.) 
# """

from keras.api.models import Sequential
from keras.api.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.api.layers import Convolution2D, MaxPooling2D
from keras.api.optimizers import SGD
from keras.api.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
import numpy as np
import keras 
import keras.api.backend as K
from keras.api.callbacks import LearningRateScheduler
import math
from keras import callbacks
import glob
from PIL import Image
from keras.src.utils import plot_model
import h5py
import time


## Basic functions 

In [5]:
# It is good to know the pid of the running code in case you need to stop  or monitor. 
# print (os.getpid())
import keras.api

file_open = lambda x,y: glob.glob(os.path.join(x,y))

# learning rate schedule. It is helpful when the learning rate can be dynamically set up. We will be using the callback functionality that keras provides. 
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.3
    epochs_drop = 30.0
    # This function doesn't actually affect the learning rate too much until a higher number of epochs is reached (around 30)
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    #print("Learning rate:", lrate)
    return lrate

# The following function will be used to give a number of the parameters in our model. Useful when we need to get an estimate of what size of dataset we have to use.  
def size(model): 
    return sum([np.prod(K.get_value(w).shape) for w in model.trainable_weights])

def createmodel(img_rows, img_cols, optimizer, loss, activation):
    """
    Creates a keras model

    :param img_rows: vertical height of the image in pixels
    :param img_cols: horizontal width of the image in pixels
    :param optimizer: a keras.optimizers object that the model will recieve during compilation (could also be a string)
    :param loss: a keras.losses object that the model will recieve during compilation (could also be a string)
    :param activation: a keras.layers.activation object that will be used in the model's construction
    :return: a sequential keras model
    """


    # This is a Sequential model. Graph models can be used in order to create more complex networks. 
    # Teaching Points:
    # 1. Here we utilize the adam optimization algorithm. In order to use the SGD algorithm one could replace the {adam=keras.optimizers.Adadelta(lr=0)} line with  {sgd = SGD(lr=0.0, momentum=0.9, decay=0.0, nesterov=False)} make sure you import the correct optimizer from keras. 
    # 2. This is a binary classification problem so make sure that the correct activation loss function combination is used. For such a problem the sigmoid activation function with the binary cross entropy loss is a good option
    # 3. Since this is a binary problem use   model.add(Dense(1)) NOT 2...
    # 4. For multi class model this code can be easily modified by selecting the softmax as activation function and the categorical cross entropy as loss 

    model = Sequential()
    model.add(Convolution2D(16, 3, 3, padding='same',input_shape=(img_rows, img_cols, 1)))
    model.add(activation)
    model.add(Convolution2D(16, 5, 5, padding='same'))
    model.add(BatchNormalization())
    model.add(activation)
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Convolution2D(32, 3, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(activation)
    model.add(Convolution2D(64, 5, 5, padding='same'))
    model.add(BatchNormalization())
    model.add(activation)
    model.add(Convolution2D(64, 3, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(activation)
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Convolution2D(128, 3, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(activation)
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Flatten())
    model.add(Dense(128, kernel_initializer='he_normal'))
    model.add(activation)
    model.add(Dropout(0.5)) 
    model.add(Dense(32, kernel_initializer='he_normal'))
    model.add(activation)
    model.add(Dropout(0.5)) 
    model.add(Dense(1))

    model.add(Activation('sigmoid'))

    # learning schedule callback
    
    # Original code had the variable named "adam", but the selected optimizer was adadelta (they are similar optimizers but different slightly)
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    
    # print(model.summary())
    return model

def shuffle(X, y):
    """
    Shuffles the given arrays

    :param X: images
    :param y: labels
    :return: tuple (X, y) that has been shuffled
    """
    perm = np.random.permutation(len(y))
    X = X[perm]
    y = y[perm]
    return X, y

def read_data(image):
    """
    Opens image and converts it to a m*n matrix

    :param image: image file
    :return: matrix of pixels
    """
    image = Image.open(image)
    image = image.getdata()

    image = np.array(image)
    return image.reshape(-1)

def createTrainTestValset(image_dir1, image_dir2):
    """
    Creates the train and test values from the given image directories

    :param image_dir1: directory of jpg images for class 1
    :param image_dir2: directory of jpg images for class 2
    :return: training and testing images/labels
    """
    Class1_images = file_open(image_dir1,"*.jpg")
    Class2_images = file_open(image_dir2,"*.jpg")

    # Read all the files, and create numpy arrays. 
    Class1_set = np.array([read_data(image) for image in Class1_images])
    Class2_set = np.array([read_data(image) for image in Class2_images])
    X = np.vstack((Class1_set, Class2_set))
    
    X = X / 255.0

    yclass1 = np.zeros((np.shape(Class1_set)[0]))
    yclass2 = np.ones((np.shape(Class2_set)[0]))
    
    y = np.concatenate((yclass1, yclass2))
    
    X,y = shuffle(X, y)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, y_train, X_val, y_val 

# Read the images; and split them in three different sets. 
def trainandpredict(X_train, y_train, X_val, y_val, optimizer, loss, lrate, activation=Activation(keras.activations.relu), batch_size=64, nb_epoch=5):
    """
    Train the model using some of the inputs, predict the remainder of the inputs using the fitted model and print the report.

    :param x_train: training images
    :param y_train: training labels
    :param X_val: test images
    :param y_val: test labels
    :param optimizer: a keras.optimizers object that the model will recieve during compilation (could also be a string)
    :param loss: a keras.losses object that the model will recieve during compilation (could also be a string)
    :param lrate: a LearningRateScheduler that the model will consider during fitting
    :param activation: a keras.layers.activation object that will be used in the model's construction
    :param batch_size: batch size used during fitting
    :param nb_epoch: number of epochs run during fitting
    """
    img_rows = 32
    img_cols = 32
    
    modeleval = createmodel(img_rows, img_cols, optimizer, loss, activation)

    X_train = X_train.reshape(
      -1,  # number of samples, -1 makes it so that this number is determined automatically
      img_rows,  # first image dimension (vertical)
      img_cols,  # second image dimension (horizontal)
      1,   # 1 color channel, since images are only black and white
    )
    X_val = X_val.reshape(
      -1,  # number of samples, -1 makes it so that this number is determined automatically
      img_rows,  # first image dimension (vertical)
      img_cols,  # second image dimension (horizontal)
      1,   # 1 color channel, since images are only black and white
    )

    # filepath = 'Final.weights.h5'

    # Callbacks (Don't need them for this project, but very useful for saving weights for later use)
    # best_model = ModelCheckpoint(filepath, verbose=1, monitor='val_loss',save_best_only=True,save_weights_only=True)

    # try:
    #   modeleval.load_weights(filepath)
    # except FileNotFoundError:
    #   print(f"Could not find file: {filepath}, assuming this is the first time with this model and will create a new file")
    # except ValueError as e:
    #   print(e)
    #   print("!!!!!!!ValueError detected, assuming this is a new model and a filepath for a different model's weights was inputted, consider a new weights file")
    #   sys.exit()

    # modeleval.fit(X_train, y_train,batch_size=batch_size,epochs=nb_epoch,validation_split=0.1,callbacks=[best_model,lrate],shuffle=True, verbose=0)

    start = time.time()

    modeleval.fit(X_train, y_train,batch_size=batch_size,epochs=nb_epoch,validation_split=0.1,callbacks=[lrate],shuffle=True, verbose=0)

    print("Total time to fit:", time.time() - start)

    # Some evaluation Just the basic stuff... 
    Y_cv_pred = modeleval.predict(X_val, batch_size = 32)
    roc = roc_auc_score(y_val, Y_cv_pred)
    print("ROC:", roc)

    Y_cv_pred[Y_cv_pred>=.5]=1
    Y_cv_pred[Y_cv_pred<.5]=0
    
    target_names = ['class 0', 'class 1']
    # Default notebook output size might not show all information from the result, make sure to expand it or change a setting when viewing
    print(classification_report(y_val, Y_cv_pred, target_names=target_names,digits=4))





## Run Program 

In [7]:
if __name__ == '__main__':
    import warnings
    warnings.filterwarnings("ignore")

    CurrentDir = os.getcwd()
    image_dir1 = os.path.abspath(os.path.join(os.path.abspath(os.path.join(CurrentDir, os.pardir)), "Data", "negative_images"))
    image_dir2 = os.path.abspath(os.path.join(os.path.abspath(os.path.join(CurrentDir, os.pardir)), "Data", "positive_images"))

    X_train, y_train, X_val, y_val = createTrainTestValset(image_dir1, image_dir2)

    # Part 1

    optimizers = [keras.optimizers.Adadelta, keras.optimizers.Adafactor, keras.optimizers.Adagrad, keras.optimizers.Adam, keras.optimizers.Adamax,
                  keras.optimizers.AdamW, keras.optimizers.Ftrl, keras.optimizers.Lion, keras.optimizers.Nadam, keras.optimizers.RMSprop, 
                  keras.optimizers.SGD]
    
    losses = [keras.losses.BinaryCrossentropy, keras.losses.BinaryFocalCrossentropy, keras.losses.SquaredHinge, keras.losses.Dice, 
              keras.losses.Huber, keras.losses.KLDivergence, keras.losses.MeanAbsoluteError]

    # Simple grid search
    for optimizer in optimizers: 
        for loss in losses: 
            print(optimizer)
            print(loss)
            trainandpredict(X_train, y_train, X_val, y_val,
                            optimizer=optimizer(learning_rate=0.0), 
                            loss=loss,
                            # This step_decay function doesn't actually change the learning rate until
                            # the number of epochs reaches about 30, a number of epochs we are not testing.
                            # For all intents and purposes the learning rate is essentially constant at 0.01.
                            lrate=LearningRateScheduler(step_decay))
            

    # Part 2 

    # Selected manually from looking at part 1 results
    best_combinations = [(keras.optimizers.Nadam, keras.losses.Huber), (keras.optimizers.RMSprop, keras.losses.Huber),
                         (keras.optimizers.AdamW, keras.losses.Huber), (keras.optimizers.Nadam, keras.losses.SquaredHinge),
                         (keras.optimizers.Adamax, keras.losses.BinaryCrossentropy)]

    # # Testing activation functions
    activations = [Activation(keras.activations.relu), Activation(keras.activations.selu), Activation(keras.activations.sigmoid)]

    for comb in best_combinations:
        for a in activations:
            print("optimizer:", comb[0])
            print("loss:", comb[1])
            print("act:", a.activation)
            trainandpredict(X_train, y_train, X_val, y_val,
                    optimizer=comb[0](learning_rate=0.0), 
                    loss=comb[1],
                    lrate=LearningRateScheduler(step_decay),
                    activation=a)
            
    # Testing batch sizes
    batch_sizes = [32, 64, 128]
    for comb in best_combinations:
        for b in batch_sizes:
            print("optimizer:", comb[0])
            print("loss:", comb[1])
            print("batch_size:", b)
            trainandpredict(X_train, y_train, X_val, y_val,
                    optimizer=comb[0](learning_rate=0.0), 
                    loss=comb[1],
                    lrate=LearningRateScheduler(step_decay),
                    batch_size=b)
            
    # Testing epochs
    epochs = [5, 10, 15]
    for comb in best_combinations:
        for e in epochs:
            print("optimizer:", comb[0])
            print("loss:", comb[1])
            print("epochs:", e)
            trainandpredict(X_train, y_train, X_val, y_val,
                    optimizer=comb[0](learning_rate=0.0), 
                    loss=comb[1],
                    lrate=LearningRateScheduler(step_decay),
                    nb_epoch=e)

    
    
        
    

<class 'keras.src.optimizers.adadelta.Adadelta'>
<class 'keras.src.losses.losses.BinaryCrossentropy'>
Total time to fit: 10.79953932762146
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
ROC: 0.664991963928357
              precision    recall  f1-score   support

     class 0     0.4637    0.4487    0.4560       526
     class 1     0.7648    0.7755    0.7701      1216

    accuracy                         0.6768      1742
   macro avg     0.6142    0.6121    0.6131      1742
weighted avg     0.6739    0.6768    0.6753      1742

